In [2]:
import requests
import numpy as np
import json
import pandas as pd
from matplotlib import pyplot as plt
import os
import plotly.graph_objects as go
from ipywidgets import widgets
import plotly.express as px
key = 'MD9WC-r9IQ61zOa3fuqKSbw4TT94t4-pMjUC'
## Note: Traffic counts taken in June, July and August can be 
## lower than average due to local school and University closures.

In [21]:
def load_clean(path):
    df = pd.read_csv(path)
    df['start_time'] = pd.to_datetime(df['start_time'], utc=True)
    return df

def load_clean_all(csv_paths):
    return {path.strip(".csv"): load_clean(path) for path in csv_paths}

def time_sorter(dfs_dict):
    df_hourly = pd.DataFrame()   
    for k in dfs_dict:
        v = dfs_dict[k]
        x = v.groupby(v['start_time'].dt.hour)['record_id'].count()
        df_hourly[f'{k}'] = x
    return df_hourly

def time_day_sort(dfs_dict):
    df_td = pd.DataFrame()   
    for k in dfs_dict:
        v = dfs_dict[k]
        x = v.groupby([v['day_of_week'],v['start_time'].dt.hour])['record_id'].count()
        df_td[f'{k}'] = x
    return df_td

def traffic_percent(df):
    z = df
    z1 = df.sum()
    day_dict = {'Friday':z1[0], 'Monday':z1[1], 'Saturday':z1[2], 'Sunday':z1[3], 'Thursday':z1[4], 'Tuesday':z1[5],
            'Wednesday':z1[6]}
    for col in df:
        total = day_dict[col]
        z[col] = round(z[col]/total*100, 2)
    return z

def time_day_sort_pct(dfs_dict):
    df_td = pd.DataFrame([i for i in range(0,24)])  
    for k in dfs_dict:
        df_name = f'{k}'
        v = dfs_dict[k]
        x = pd.DataFrame(v.groupby([v['day_of_week'],v['start_time'].dt.hour])['record_id'].count()).unstack(level = 0).droplevel(axis=1, level=0)
        x1 = traffic_percent(x)
        df_td = df_td.merge(x1, how = "outer", right_on = x1.index, left_on = df_td.index)
        df_td = df_td.drop(columns = 'key_0')
    return df_td

def col_namer(df):
    names = ["0"]
    roads = ['183','35', 'lamar', 'burnet','2222' ]
    dow = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
    start = 1
    a = list(df)[start:start+7]
    for road in roads:
        b = [ f'{road}_' + i for i in dow]
        names.extend(b)
    return names

### Loading dfs for 2017, 2018, and 2019, cleaning and converting counts to percentages 

In [24]:
csv_paths = [file for file in os.listdir() if file.endswith("csv")]
csv_paths17 = ["df183_17.csv", "df35_17.csv", 'dflamar_17.csv', 'dfburnet_17.csv', 'df2222_17.csv']
csv_paths18 = ["df183_18.csv", "df35_18.csv", 'dflamar_18.csv', 'dfburnet_18.csv', 'df2222_18.csv']
csv_paths19 = ['df35_19.csv', 'df183_19.csv', 'dflamar_19.csv', 'dfburnet_19.csv', 'df2222_19.csv']

dfs17 = load_clean_all(csv_paths17)
dfs18 = load_clean_all(csv_paths18)
dfs19 = load_clean_all(csv_paths19)

df17 = time_day_sort_pct(dfs17)
df18 = time_day_sort_pct(dfs18)
df19 = time_day_sort_pct(dfs19)

cols17 = col_namer(df17)
cols18 = col_namer(df18)
cols19 = col_namer(df19)

df17.columns = cols17
df18.columns = cols18
df19.columns = cols19

df17.to_csv("extra/df17.csv")
df18.to_csv("extra/df18.csv")
df19.to_csv("extra/df19.csv")


<ipython-input-24-acb7073171dc>:6: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.



### Loading and converting traffic incident data for plotting

In [12]:
incident_df = pd.read_csv("traffic_incidents.csv")
incident_df['time'] = pd.to_datetime(incident_df['traffic_report_status_date_time'])
incident_df['year'] = incident_df['time'].dt.year
incident_df['day'] = incident_df['time'].dt.dayofweek
incident_df['hour'] = incident_df['time'].dt.hour

In [221]:
dow_dict = {0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}
dow_int = incident_df['day']
dow = [dow_dict[row] for row in dow_int]
incident_df['day_of_week'] = dow
dfi_hour_day = pd.DataFrame(incident_df.groupby([incident_df['day_of_week'],incident_df['hour']])['traffic_report_id'].count()).unstack(level = 0).droplevel(axis = 1, level = 0)
dfi_percent = incident_percent(dfi_hour_day)
dfi_percent.to_csv("dfi_percent.csv")
